In [ ]:
from IPython.display import Image, display
from typing import TypedDict

from langgraph.graph import StateGraph, START, END

# 1. Agent state

In [ ]:
class AgentState(TypedDict):
    number1: int
    number2: int
    operation: str
    final_number: int

# 2. Create nodes

In [ ]:
def adder(state: AgentState) -> AgentState:
    """This node adds the 2 numbers"""
    state["final_number"] = state["number1"] + state["number2"]
    return state


def subtractor(state: AgentState) -> AgentState:
    """This node subtracts the 2 numbers"""
    state["final_number"] = state["number1"] - state["number2"]
    return state


def decide_next_node(state: AgentState) -> AgentState:
    """This node will select the next phase"""
    if state["operation"] == "+":
        return "addition_operation"

    elif state["operation"] == "-":
        return "subtraction_operation"

# 3. Create graph

In [ ]:
graph = StateGraph(AgentState)

graph.add_node("add_node", adder)
graph.add_node("subtract_node", subtractor)
graph.add_node("router", lambda state: state)

graph.add_edge(START, "router")
graph.add_conditional_edges(
    "router",
    decide_next_node,
    {
        "addition_operation": "add_node",
        "subtraction_operation": "subtract_node",
    },
)

graph.add_edge("add_node", END)
graph.add_edge("subtract_node", END)

app = graph.compile()

# 4. Visualize graph

In [ ]:
display(Image(app.get_graph().draw_mermaid_png()))

# 5. Invoke

In [ ]:
initial_state = AgentState(number1=10, operation="-", number2=5)

result = app.invoke(initial_state)

print("Final Result:", result["final_number"])

In [ ]:
initial_state = AgentState(number1=10, operation="+", number2=5)

result = app.invoke(initial_state)

print("Final Result:", result["final_number"])